# When do triggers occur?

When multiple triggers occur (e.g. when doing a [segmented capture](segmented.ipynb)), it can be really useful to learn when the triggers occur (relative to the first trigger).

`scope.trigger.get_trigger_times()` lets you do just that.

We use it in our [sca205](https://github.com/newaetech/chipwhisperer-jupyter/tree/main/courses/sca205) notebooks and in SADExplorer (TODO-link), and it's a critical part of both of those; here we show how easy it is to use with a simple example.

**Supported Capture Hardware:**\
❌ CW-Nano\
❌ CW-Lite\
❌ CW-Pro\
✅ CW-Husky

**Required ChipWhisperer software installation:**\
✅ any release

In [1]:
%run 'connect.ipynb'
cw.scope_logger.setLevel(cw.logging.WARNING) # turn warnings back on

OSError: Could not find ChipWhisperer. Is it connected?

OSError: Could not find ChipWhisperer. Is it connected?

There are different ways to get multiple triggers. With the `simpleserial-aes` firmware, the easiest is to use the `'n'` and `'f'` commands so that sending a single plaintext triggers multiple encryptions, each of which raises and lowers the `TIO4` trigger line.

Let's trigger 4 encryptions:

In [ ]:
NUM_ENCRYPTIONS = 4
scope.adc.samples = 98000
target.simpleserial_write('n', list(int.to_bytes(NUM_ENCRYPTIONS, length=2, byteorder='big'))) # set the number of encryptions
target.simpleserial_wait_ack()

scope.arm()
target.simpleserial_write('f', bytearray(16)) # trigger the encryptions
ret = scope.capture(poll_done=True)
wave = scope.get_last_trace()
resp = target.simpleserial_read('r', 16)

Note that here a "segmenting error" is normal and expected because triggers occurs while the capture is still ongoing.

In [ ]:
scope.adc.errors

In [ ]:
scope.errors.clear()

Now we read the timestamps for the triggers (relative to the first trigger). Since the target triggered 4 times, we should get 3 timestamps, and they should all be evenly spaced.

Timestamps are measured with the ADC sampling clock.

In [ ]:
ttimes = scope.trigger.get_trigger_times()

In [ ]:
print(ttimes)

In [ ]:
assert len(ttimes) == 3
for t in range(1, len(ttimes)):
    assert ttimes[t] == ttimes[0]

While this example is not terribly helpful, you could adapt it to learn, for example, when a particular portion of the AES round occurs, by raising the trigger line every time (for example) that the mixcolumns operation is executed.

In [ ]:
assert not scope.adc.errors, scope.adc.errors